1 лабораторная работа - "word2vec и TF-IDF". БВТ2202 Лесовой Роман Сергеевич

In [1]:
!python.exe -m pip install --upgrade pip
import pandas as pd
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from nltk import sent_tokenize
import os
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\roman\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\roman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\roman\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
def clean_text(text):
    """
    Функция для очистки текста
    """
    # Инициализация инструментов
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))  # Стоп-слова на английском

    # Удаляем римские цифры
    text = re.sub(r'\b[IVXLCDM]+\b', '', text, flags=re.IGNORECASE)
    # Удаляем обычные цифры
    text = re.sub(r'd+', '', text)
    # Удаляем _
    text = re.sub('_', '', text)

    # Токенизация текста
    tokens = word_tokenize(text)
    # Удаляем стоп-слова и проводим лемматизацию (приведение к базовой форме слова)
    cleaned_tokens = [
        lemmatizer.lemmatize(token.lower())  # Лемматизация и приведение к нижнему регистру
        for token in tokens
        if token.isalpha() and token.lower() not in stop_words  # Убираем неалфавитные символы и стоп-слова
    ]

    return ' '.join(cleaned_tokens)


def process_files(directory_path):
    """
    Функция читает все .txt файлы из указанной директории,
    выполняет очистку текста, фильтрует абзацы по длине,
    применяет к тексту функцию clean_text
    """
    # Для TF-IDF
    list_of_paragraphs = []
    # Для Word2Vec
    list_of_sentences = []

    # Обходим все файлы в директории
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()

                # Разделяем на абзацы
                paragraphs = text.split('\n\n')
                paragraphs = [p.strip() for p in paragraphs if p.strip()]

                # Очищаем каждый абзац
                cleaned_paragraphs = []
                for paragraph in paragraphs:
                    cleaned_paragraph = clean_text(paragraph)  # Очистка текста
                    # Не берём слишком короткие абзацы
                    if len(cleaned_paragraph) >= 100:
                        cleaned_paragraphs.append(cleaned_paragraph)

                list_of_paragraphs.extend(cleaned_paragraphs)

                # Разделяем текст на предложения для Word2Vec
                # Используем NLTK для разделения на предложения
                sentences = sent_tokenize(text)
                for sentence in sentences:
                    cleaned_sentence = clean_text(sentence)  # Очистка текста
                    tokenized_sentence = word_tokenize(cleaned_sentence)  # Токенизация предложения
                    if tokenized_sentence:  # Проверяем, что предложение не пустое после очистки
                        list_of_sentences.append(tokenized_sentence)

    return list_of_paragraphs, list_of_sentences



# Путь к текстам
directory_path = "texts/"

# Данные для TF-IDF и Word2Vec
paragraphs_for_tfidf, sentences_for_word2vec = process_files(directory_path)

# Вывод полученных данных
print("Количество абзацев TF-IDF:", len(paragraphs_for_tfidf))
print("Пример абзаца:", paragraphs_for_tfidf[0])
print("\nКоличество предложений для Word2Vec:", len(sentences_for_word2vec))
print("Пример предложения:", sentences_for_word2vec[0])

# создание TF-IDF модели
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(paragraphs_for_tfidf)
print("\nTF-IDF матрица создана. Размер матрицы:", tfidf_matrix.shape)

# обучение Word2Vec модели
from gensim.models import Word2Vec

word2vec_model = Word2Vec(sentences=sentences_for_word2vec, vector_size=100, window=5, min_count=1, workers=4)
print("\nWord2Vec модель обучена.")

Количество абзацев TF-IDF: 21906
Пример абзаца: sai name though common name efinition corresponing name iffers thus real man figure picture lay claim name yet equivocally name though common name efinition corresponing name iffers shoul one efine sense animal efinition one case appropriate case

Количество предложений для Word2Vec: 127111
Пример предложения: ['sai', 'name', 'though', 'common', 'name', 'efinition', 'corresponing', 'name', 'iffers']

TF-IDF матрица создана. Размер матрицы: (21906, 37105)

Word2Vec модель обучена.


In [3]:
# первые 10 слов и их TF-IDF значения для первого абзаца:

# Получаем список всех слов (фичей)
feature_names = tfidf_vectorizer.get_feature_names_out()
# Берем первый абзац из TF-IDF матрицы
first_paragraph_tfidf = tfidf_matrix[0]
# Преобразуем в массив
first_paragraph_tfidf_dense = first_paragraph_tfidf.toarray()[0]

# Сопоставляем слова с их TF-IDF значениями
word_tfidf_pairs = [(feature_names[i], first_paragraph_tfidf_dense[i]) for i in range(len(feature_names)) if first_paragraph_tfidf_dense[i] > 0]
sorted_word_tfidf_pairs = sorted(word_tfidf_pairs, key=lambda x: x[1], reverse=True)

# топ 10 слов с наибольшими значениями TF-IDF
print("\nТоп-10 слов с наибольшими TF-IDF значениями в первом абзаце:")
for word, score in sorted_word_tfidf_pairs[:10]:
    print(f"{word}: {score:.4f}")


Топ-10 слов с наибольшими TF-IDF значениями в первом абзаце:
name: 0.6218
efinition: 0.4115
iffers: 0.3055
corresponing: 0.2831
equivocally: 0.2220
common: 0.1609
case: 0.1502
appropriate: 0.1463
picture: 0.1389
efine: 0.1345


In [5]:
def print_similar(_similar_words, _max = 3, none_words = []):
    # это список кортежей, где каждый кортеж содержит слово и его косинусное сходство
    i = 0
    for _word, _similarity in _similar_words:
        if _word not in none_words:
            i+=1
            print(f"{_word}: {_similarity:.4f}")
            if i >= _max:
                break


def get_result_vector(model, add_words=[], substract_words=[]):
    result_vector = np.zeros(model.vector_size)
    for word in add_words:
        if word in model.wv:
            result_vector += model.wv[word]
    for word in substract_words:
        if word in model.wv:
            result_vector -= model.wv[word]

    return result_vector

def get_similar_words(model, add_words=[], substract_words=[]):
    if len(substract_words) > 0:
        print(" + ".join(add_words) + " - " + " - ".join(substract_words))
    else:
        print(" + ".join(add_words))
    _result_vector = get_result_vector(model, add_words, substract_words)
    _similar_words = word2vec_model.wv.similar_by_vector(_result_vector)
    print_similar(_similar_words, none_words=add_words+substract_words)
    print()

get_similar_words(word2vec_model, ["order", "people"], ["chaos"])
get_similar_words(word2vec_model, ["rage", "anger", "violence"], ["good"])
get_similar_words(word2vec_model, ["king", "woman"], ["man"])
get_similar_words(word2vec_model, ["defeat", "war"])

order + people - chaos
nation: 0.7072
men: 0.6405
mankin: 0.6371

rage + anger + violence - good
expose: 0.8865
terror: 0.8671
violent: 0.8605

king + woman - man
queen: 0.7351
henry: 0.7266
pope: 0.6806

defeat + war
peace: 0.8152
conquest: 0.8054
wage: 0.7647

